# Library load

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn import metrics

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb

# Data load

In [2]:
train_df = pd.read_csv("../input/dacon-apartment-price-prediction/train.csv")
test_df = pd.read_csv("../input/dacon-apartment-price-prediction/test.csv")
park = pd.read_csv("../input/dacon-apartment-price-prediction/park.csv")
day_care = pd.read_csv("../input/dacon-apartment-price-prediction/day_care_center.csv")

# 데이터 확인하기

In [3]:
train_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,transaction_real_price
0,0,7622,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002,200801,21~31,2,37500
1,1,5399,서울특별시,필운동,142,사직파크맨션,필운동 142 사직파크맨션,99.17,1973,200801,1~10,6,20000
2,2,3578,서울특별시,필운동,174-1,두레엘리시안,필운동 174-1 두레엘리시안,84.74,2007,200801,1~10,6,38500
3,3,10957,서울특별시,내수동,95,파크팰리스,내수동 95 파크팰리스,146.39,2003,200801,11~20,15,118000
4,4,10639,서울특별시,내수동,110-15,킹스매너,내수동 110-15 킹스매너,194.43,2004,200801,21~31,3,120000


In [4]:
print(len(train_df))
print(len(test_df))

1216553
5463


In [5]:
train_df.isnull().sum()

transaction_id            0
apartment_id              0
city                      0
dong                      0
jibun                     0
apt                       0
addr_kr                   0
exclusive_use_area        0
year_of_completion        0
transaction_year_month    0
transaction_date          0
floor                     0
transaction_real_price    0
dtype: int64

In [6]:
test_df.isnull().sum()

transaction_id            0
apartment_id              0
city                      0
dong                      0
jibun                     0
apt                       0
addr_kr                   0
exclusive_use_area        0
year_of_completion        0
transaction_year_month    0
transaction_date          0
floor                     0
dtype: int64

In [7]:
park.head()

,city,gu,dong,park_name,park_type,park_area,park_exercise_facility,park_entertainment_facility,park_benefit_facility,park_cultural_facitiy,park_facility_other,park_open_year,reference_date
0,부산광역시,남구,대연동,대연공원,근린공원,55227.0,NaN,NaN,NaN,NaN,NaN,1944.0,2018-08-14
1,부산광역시,남구,대연동,당곡공원,역사공원,75465.0,NaN,NaN,NaN,NaN,NaN,1944.0,2018-08-14
2,부산광역시,남구,대연동,대일어린이공원,어린이공원,422.1,NaN,NaN,NaN,NaN,NaN,1965.0,2018-08-14
3,부산광역시,남구,대연동,못골어린이공원,어린이공원,499.8,NaN,NaN,NaN,NaN,NaN,1968.0,2018-08-14
4,부산광역시,남구,용호동,솔밭어린이공원,어린이공원,1500.2,NaN,NaN,NaN,NaN,NaN,1970.0,2018-08-14


In [8]:
len(park)

1359

In [9]:
park.isnull().sum()

city                              0
gu                                3
dong                              0
park_name                         0
park_type                         0
park_area                         0
park_exercise_facility         1082
park_entertainment_facility     924
park_benefit_facility          1093
park_cultural_facitiy          1287
park_facility_other            1184
park_open_year                  422
reference_date                    0
dtype: int64

In [10]:
day_care.head()

,city,gu,day_care_name,day_care_type,day_care_baby_num,teacher_num,nursing_room_num,playground_num,CCTV_num,is_commuting_vehicle,reference_date
0,서울특별시,서대문구,가람어린이집,국공립,70,11.0,5.0,1.0,7.0,N,2018-07-31
1,서울특별시,서대문구,가좌제일어린이집,국공립,88,22.0,5.0,2.0,10.0,N,2018-07-31
2,서울특별시,서대문구,경찰청어린이집,직장,90,23.0,6.0,1.0,32.0,N,2018-07-31
3,서울특별시,서대문구,고운햇살어린이집,가정,18,5.0,3.0,0.0,4.0,N,2018-07-31
4,서울특별시,서대문구,고은어린이집,국공립,93,20.0,7.0,2.0,10.0,N,2018-07-31


In [11]:
len(day_care)

7551

In [12]:
day_care.isnull().sum()

city                       0
gu                         0
day_care_name              0
day_care_type              0
day_care_baby_num          0
teacher_num              225
nursing_room_num        3199
playground_num          3925
CCTV_num                2271
is_commuting_vehicle     496
reference_date             0
dtype: int64

# Preprocessing

In [13]:
# 아파트 실거래가(예측변수) 추출
y = train_df['transaction_real_price']

In [14]:
train_df.drop('transaction_real_price',axis= 1,inplace =True)

In [15]:
# 트레인셋 라벨링
train_df['train_YN']=np.ones(len(train_df))
test_df['train_YN']=np.zeros(len(test_df))

In [16]:
# 트레인셋과 테스트셋 결합 후 전처리 
df = pd.concat([train_df,test_df],axis =0)

In [17]:
df.shape

(1222016, 13)

In [18]:
# 거래날짜 변수 유니크 값 확인
df['transaction_date'].unique()

array(['21~31', '1~10', '11~20', '21~29', '21~30', '21~28'], dtype=object)

In [19]:
# 거래날짜 변수 값 변형
df.loc[df['transaction_date']=='21~31', 'transaction_date'] = 2
df.loc[df['transaction_date']=='1~10', 'transaction_date'] = 0
df.loc[df['transaction_date']=='11~20', 'transaction_date'] = 1
df.loc[df['transaction_date']=='21~29', 'transaction_date'] = 2
df.loc[df['transaction_date']=='21~30', 'transaction_date'] = 2
df.loc[df['transaction_date']=='21~28', 'transaction_date'] = 2

In [20]:
# 거래년월 분리 후 '연도' 와 '월' 컬럼 생성
df['year'] = df['transaction_year_month'].apply(lambda x : int(str(x)[:4]))
df['month'] = df['transaction_year_month'].apply(lambda x : int(str(x)[4:]))

In [21]:
# '월' 변수로 '분기' 변수 생성
quarter = np.array(df['month']).tolist()

In [22]:
# 1~4분기로 나누는 조건문
for i in range(len(quarter)):
    if quarter[i] in [1,2,3]:
        quarter[i] = '1quarter'
    elif quarter[i] in [4,5,6]:
        quarter[i] = '2quarter'
    elif quarter[i] in [7,8,9]:
        quarter[i]  = '3quarter'
    else :
        quarter[i] = '4quarter'

In [23]:
# 데이터 셋에 '분기' 변수 생성
df['quarter'] = pd.DataFrame(quarter,columns=['quarter'])

In [24]:
# '분기'변수를 숫자로 변환
df.loc[df['quarter'] == '1quarter', 'quarter'] = 1
df.loc[df['quarter'] == '2quarter', 'quarter'] = 2
df.loc[df['quarter'] == '3quarter', 'quarter'] = 3
df.loc[df['quarter'] == '4quarter', 'quarter'] = 4


In [25]:
# 결측값 여부 확인 
df.isnull().sum()

transaction_id            0
apartment_id              0
city                      0
dong                      0
jibun                     0
apt                       0
addr_kr                   0
exclusive_use_area        0
year_of_completion        0
transaction_year_month    0
transaction_date          0
floor                     0
train_YN                  0
year                      0
month                     0
quarter                   0
dtype: int64

In [26]:
# 공원 데이터셋에서 '동' 유니크 개수 확인
park['dong'].nunique()

237

In [27]:
# 시, 구, 동을 그룹화 하여 공원 개수 카운트 
park_count = park.groupby([park['city'],park['gu'],park['dong']]).count()['park_name']

In [28]:
# 시, 구, 동을 그룹화 한뒤 동마다 공원의 면적과 개장연도를 합산 후 공원 개수 카운트 데이터와 concat
park_df = pd.concat([park.groupby([park['city'],park['gu'],park['dong']]).sum(),park_count],axis =1)

In [29]:
park_df.head()

park_area  park_open_year  park_name
city  gu  dong                                      
부산광역시 강서구 녹산동     12211.9          2010.0          1
          대저1동      646.0          2013.0          1
          명지동    348482.8         50133.0         25
          송정동     57560.0          3984.0          2
          신호동     69323.0          7982.0          4

In [30]:
# 컬럼명 변경 
park_df.columns = ['park_area','park_open_year','count']

In [31]:
# 개장년도는 활용 불가능하다고 판단되어 삭제 
park_df.drop('park_open_year',axis=1,inplace = True)

In [32]:
# 공원 개당 평균 면적 계산 후 데이터 셋에 합함  
park_df['park_area_mean'] = park_df['park_area']/park_df['count']

In [33]:
# 그룹화 한 데이터 셋에서 (시,구,동) 분리
zz = []
dong_gu_name = []
for idx, dong_gu in enumerate(park_df.index):
    zz.append(idx)
    dong_gu_name.append(dong_gu)

In [34]:
# 위에서 분리한 시, 구 , 동을 다시 각각 새로운 리스트로 할당
dong = []
gu = []
city =[]
for i in range(len(dong_gu_name)):
    city.append(dong_gu_name[i][0])
    dong.append(dong_gu_name[i][2])
    gu.append(dong_gu_name[i][1])

In [35]:
# 리스트를 데이터프레임 화 
dong_df = pd.DataFrame(dong,columns=['dong'])
gu_df = pd.DataFrame(gu,columns=['gu'])
city_df = pd.DataFrame(city,columns=['city'])

In [36]:
# 인덱스 번호를 맞춘 후 합치기 위해 재지정
park_df.index=range(len(park_df))

In [37]:
# 데이터 프레임에 시,구,동 합치기
park_df = pd.concat([park_df,dong_df],axis=1)
park_df = pd.concat([park_df,gu_df],axis=1)
park_df = pd.concat([park_df,city_df],axis=1)

In [38]:
# 트레인 셋과 합치기 위해 키 값으로 사용할 동이 고유 값인지 확인
print(park_df['dong'].nunique()) 
print(park_df['park_area'].nunique())
print(park_df['park_area_mean'].nunique())

237
244
244


#### 같은 동의 이름이 있다고 의심이 됨

In [39]:
# 같은 동의 이름을 탐색
for i in range(len(park_df)):
    for j in range(len(park_df)):
        if park_df['dong'][i] == park_df['dong'][j] and j>i :
            print(park_df['dong'][i])

송정동
송정동
청룡동
송정동
삼성동
신사동
신사동
신사동
신원동


In [40]:
# 삼성동, 송정동, 신사동, 신원동, 청룡동이 이름이 같은 동인 것을 확인하였고 트레인셋의 조인에 영향을 주지 않기 위해 제거하려고 따로 저장 
remainder = park_df.loc[park_df['dong'].isin(['삼성동','송정동','신사동','신원동','청룡동'])]

In [41]:
# 삭제하기 위해 인덱스 추출 
delidx = remainder.index.tolist()

In [42]:
# 같은 이름의 동 삭제
park_df.drop(park_df.index[delidx],inplace=True)

### 공원데이터와 기존데이터 merge

In [43]:
df_merge = pd.merge(df,park_df,on=['dong'],how='outer')

In [44]:
len(df_merge) # 기존 데이터셋과 길이가 일치하는 것을 확인함

1222016

In [45]:
# 컬럼명 확인
df_merge.columns

Index(['transaction_id', 'apartment_id', 'city_x', 'dong', 'jibun', 'apt',
       'addr_kr', 'exclusive_use_area', 'year_of_completion',
       'transaction_year_month', 'transaction_date', 'floor', 'train_YN',
       'year', 'month', 'quarter', 'park_area', 'count', 'park_area_mean',
       'gu', 'city_y'],
      dtype='object')

In [46]:
# 중복 컬럼 제거 
del df_merge['city_y']

In [47]:
# 컬럼 재정의
df_merge.columns = ['transaction_id', 'apartment_id', 'city', 'dong', 'jibun',
       'apt', 'addr_kr', 'exclusive_use_area', 'year_of_completion',
       'transaction_year_month', 'transaction_date', 'floor', 'train_YN',
       'year', 'month', 'quarter', 'park_area', 'count', 'park_area_mean',
       'gu']

In [48]:
# 결측값 확인 
df_merge.isnull().sum()

transaction_id                 0
apartment_id                   0
city                           0
dong                           0
jibun                          0
apt                            0
addr_kr                        0
exclusive_use_area             0
year_of_completion             0
transaction_year_month         0
transaction_date               0
floor                          0
train_YN                       0
year                           0
month                          0
quarter                        0
park_area                 275911
count                     275911
park_area_mean            275911
gu                        275911
dtype: int64

In [49]:
# 결측값 0으로 대체
df_merge.fillna(0,inplace=True)

In [50]:
# 보육시설 결측값 확인
day_care.isnull().sum()

city                       0
gu                         0
day_care_name              0
day_care_type              0
day_care_baby_num          0
teacher_num              225
nursing_room_num        3199
playground_num          3925
CCTV_num                2271
is_commuting_vehicle     496
reference_date             0
dtype: int64

In [51]:
# '통학차량 운영여부' 변수를 사용하기위해 확인
day_care['is_commuting_vehicle'].unique()


array(['N', 'Y', ' ', nan, 'n', 'y'], dtype=object)

In [52]:
# 통학차량 운영여부 변수 사용하기위해 변환
day_care.loc[day_care['is_commuting_vehicle']=='N','is_commuting_vehicle'] = 0
day_care.loc[day_care['is_commuting_vehicle']=='Y','is_commuting_vehicle'] = 1
day_care.loc[day_care['is_commuting_vehicle']==' ','is_commuting_vehicle'] = 0
day_care.loc[day_care['is_commuting_vehicle']=='n','is_commuting_vehicle'] = 0
day_care.loc[day_care['is_commuting_vehicle']=='y','is_commuting_vehicle'] = 1

In [53]:
# 통학차량 운영여부 결측값 0 으로 변환
day_care['is_commuting_vehicle'].fillna(0,inplace=True)

In [54]:
# 통학차량 운영여부 변수 0,1 만 존재함을 확인
day_care['is_commuting_vehicle'].unique()

array([0, 1])

In [55]:
# 통학차량 운영여부 변수 숫자형으로 변환
day_care['is_commuting_vehicle'] = day_care['is_commuting_vehicle'].astype('int')

In [56]:
# 구와 시별로 그룹화 하여 보육시설 개수 확인 
cnt = day_care.groupby([day_care['gu'],day_care['city']]).count()['day_care_name']

In [57]:
# 구와 시별로 그룹화 하여 각 변수의 합과 위에서 계산한 보육시설 개수를 concat
daycare_df = pd.concat([day_care.groupby([day_care['gu'],day_care['city']]).sum(),cnt],axis=1)

In [58]:
# 컬럼 재정의
daycare_df.columns = ['day_care_baby_num', 'teacher_num', 'nursing_room_num',
       'playground_num', 'CCTV_num', 'is_commuting_vehicle','cnt']

In [59]:
# 보육시설 당 평균 정원수 변수 추가
daycare_df['day_care_mean'] = daycare_df['day_care_baby_num']/daycare_df['cnt']

In [60]:
# 구와 시를 데이터 셋에서 분리 
zzz = []
gu_city_name = []
for idx, gu_city in enumerate(daycare_df.index):
    zzz.append(idx)
    gu_city_name.append(gu_city)

In [61]:
# 구와 시를 또 다시 나눔 
gu = []
city = []
for i in range(len(gu_city_name)):
    gu.append(gu_city_name[i][0])
    city.append(gu_city_name[i][1])

In [62]:
# 각각 데이터 프레임화 
gu_df = pd.DataFrame(gu,columns=['gu'])
city_df = pd.DataFrame(city,columns=['city'])

In [63]:
# 인덱스 재정의 
daycare_df.index = range(len(daycare_df))

In [64]:
# 구와 시 변수를 데이터셋에 concat
daycare_df = pd.concat([daycare_df,gu_df],axis=1)
daycare_df = pd.concat([daycare_df,city_df],axis=1)

### 기존 데이터셋과 보육시설 데이터셋 merge

In [65]:
final_df = pd.merge(df_merge,daycare_df,on=['gu','city'],how='outer')

In [66]:
# merge가 제대로 됐는지 확인
print(len(final_df))
print(len(df_merge))

1222017
1222016


In [67]:
final_df.isnull().sum() # transaction_id에서 결측값 발생

transaction_id                 1
apartment_id                   1
city                           0
dong                           1
jibun                          1
apt                            1
addr_kr                        1
exclusive_use_area             1
year_of_completion             1
transaction_year_month         1
transaction_date               1
floor                          1
train_YN                       1
year                           1
month                          1
quarter                        1
park_area                      1
count                          1
park_area_mean                 1
gu                             0
day_care_baby_num         313238
teacher_num               313238
nursing_room_num          313238
playground_num            313238
CCTV_num                  313238
is_commuting_vehicle      313238
cnt                       313238
day_care_mean             313238
dtype: int64

In [68]:
# transaction_id 결측값인 행을 삭제 
final_df.drop(final_df.index[final_df.loc[final_df['transaction_id'].isnull()].index],inplace=True)

In [69]:
final_df.isnull().sum() #제거 되었음을 확인

transaction_id                 0
apartment_id                   0
city                           0
dong                           0
jibun                          0
apt                            0
addr_kr                        0
exclusive_use_area             0
year_of_completion             0
transaction_year_month         0
transaction_date               0
floor                          0
train_YN                       0
year                           0
month                          0
quarter                        0
park_area                      0
count                          0
park_area_mean                 0
gu                             0
day_care_baby_num         313238
teacher_num               313238
nursing_room_num          313238
playground_num            313238
CCTV_num                  313238
is_commuting_vehicle      313238
cnt                       313238
day_care_mean             313238
dtype: int64

In [70]:
# 모든 결측값 0으로 대체 
final_df.fillna(0,inplace=True)

In [71]:
final_df.head()

,transaction_id,apartment_id,city,dong,jibun,apt,addr_kr,exclusive_use_area,year_of_completion,transaction_year_month,transaction_date,floor,train_YN,year,month,quarter,park_area,count,park_area_mean,gu,day_care_baby_num,teacher_num,nursing_room_num,playground_num,CCTV_num,is_commuting_vehicle,cnt,day_care_mean
0,0.0,7622.0,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002.0,200801.0,2.0,2.0,1.0,2008.0,1.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,98.0,7623.0,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002.0,200803.0,1.0,2.0,1.0,2008.0,3.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,511.0,7622.0,서울특별시,신교동,6-13,신현(101동),신교동 6-13 신현(101동),84.82,2002.0,200903.0,2.0,3.0,1.0,2009.0,3.0,1.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,840.0,7623.0,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),81.02,2002.0,200909.0,0.0,1.0,1.0,2009.0,9.0,3.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1201.0,7623.0,서울특별시,신교동,6-11,신현(102동),신교동 6-11 신현(102동),84.82,2002.0,201011.0,2.0,3.0,1.0,2010.0,11.0,4.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [72]:
#트레인 셋과 테스트 셋 분리
train_df = final_df[final_df['train_YN'] == 1]

In [73]:
test_df = final_df[final_df['train_YN'] == 0]

In [74]:
# 제대로 나누어졌는지 길이 확인 
print(len(train_df)+len(test_df))
print(len(final_df))

1222016
1222016


In [75]:
# 인덱싱 재정의 
train_df = train_df.sort_values(["transaction_id"], ascending=True)

In [76]:
test_df = test_df.sort_values(["transaction_id"], ascending=True)

In [77]:
train_df.index = range(len(train_df))

In [78]:
test_df.index = range(len(test_df))

# Feature engineering

In [79]:
train_df = pd.concat([train_df,y],axis=1)

In [80]:
# aprtment_id의 유니크 개수 확인 
df['apartment_id'].nunique()

12582

### apartment_id를 기준으로 Mean Encoding 순서

1. 'apartment_id'를 그룹화하고 아파트마다 예측 변수인 '실제 거래가'의 평균을 구합니다.
2. 'apartment_id'를 그룹화하고 같은 아파트가 데이터 셋에 몇개 있는지 카운트 합니다.
3. 'transaction_real_price'의 평균을 구합니다.


### **mean encoding 기법 **

범주형 변수를 기준으로 하여 예측 변수값과 통계적인 연산을 하여 연관성을 만들어 학습이 더욱더 잘되게 해줍니다.
저는 아래에 있는 방법을 사용하였습니다.

mean encoding 변수  = (해당 아파트 평균 가격 * 해당 아파트 개수 + 아파트 실거래가의 평균) / 해당 아파트 개수

In [81]:
train_df = pd.merge(train_df,train_df.groupby(['apartment_id'])['transaction_real_price'].mean().reset_index(name='apt_id_real_price'),on='apartment_id',how='left')

In [82]:
train_df = pd.merge(train_df,train_df.groupby(['apartment_id']).size().reset_index(name='apartment_count'),on='apartment_id',how='left')

In [83]:
train_df['real_price_mean'] = train_df['transaction_real_price'].mean()

In [84]:
train_df['apartment_price_mean'] = ((train_df['apt_id_real_price'] * train_df['apartment_count']) + train_df['real_price_mean']) / train_df['apartment_count']

In [85]:
# 테스트 셋에도 mean_encoding 한 변수 추가해주기 
test_df = pd.merge(test_df,pd.concat([train_df['apartment_id'],train_df['apartment_price_mean']],axis=1).groupby(['apartment_id']).mean(),on='apartment_id',how='left')

#### 추가적으로 아파트의 연식(오래된 정도) 변수 생성

In [86]:
train_df['apt_age'] = train_df['year'] - train_df['year_of_completion']
test_df['apt_age'] = test_df['year'] - test_df['year_of_completion']

#### 부산과 서울을 1,0으로 coding

In [87]:
train_df['city']=train_df['city'].map({'서울특별시':1,'부산광역시':0})
test_df['city']=test_df['city'].map({'서울특별시':1,'부산광역시':0})

In [88]:
# Y값 추출
y = train_df['transaction_real_price']

## Correlation Check

In [89]:
# 실거래가와 전용면적
pd.concat([train_df['exclusive_use_area'],y], axis= 1).corr()

,exclusive_use_area,transaction_real_price
exclusive_use_area,1.000000,0.561563
transaction_real_price,0.561563,1.000000


In [90]:
# 실거래가와 아파트설립일자
pd.concat([y,train_df['year_of_completion']], axis= 1).corr()

,transaction_real_price,year_of_completion
transaction_real_price,1.000000,0.052656
year_of_completion,0.052656,1.000000


In [91]:
# 실거래가와 도시
pd.concat([y,train_df['city']], axis= 1).corr()

,transaction_real_price,city
transaction_real_price,1.000000,0.440886
city,0.440886,1.000000


In [92]:
# 실거래가와 거래일자(시기)
pd.concat([y,train_df['transaction_date']], axis= 1).corr()

,transaction_real_price,transaction_date
transaction_real_price,1.00000,0.01275
transaction_date,0.01275,1.00000


In [93]:
# 실거래가와 층수
pd.concat([y,train_df['floor']], axis= 1).corr()

,transaction_real_price,floor
transaction_real_price,1.000000,0.112278
floor,0.112278,1.000000


In [94]:
# 실거래가와 거래년도
pd.concat([y,train_df['year']], axis= 1).corr()

,transaction_real_price,year
transaction_real_price,1.000000,0.183714
year,0.183714,1.000000


In [95]:
# 실거래가와 거래월
pd.concat([y,train_df['month']], axis= 1).corr()

,transaction_real_price,month
transaction_real_price,1.000000,0.022886
month,0.022886,1.000000


In [96]:
# 실거래가와 해당분기
pd.concat([y,train_df['quarter']], axis= 1).corr()

,transaction_real_price,quarter
transaction_real_price,1.000000,0.024443
quarter,0.024443,1.000000


In [97]:
# 실거래가와 아파트가 위치하고 있는 동의 공원부지 넓이
pd.concat([y,train_df['park_area']], axis= 1).corr()

,transaction_real_price,park_area
transaction_real_price,1.000000,-0.066412
park_area,-0.066412,1.000000


In [98]:
# 실거래가와 아파트가 위치하고 있는 동의 공원개수 
pd.concat([y,train_df['count']], axis= 1).corr()

,transaction_real_price,count
transaction_real_price,1.000000,0.074861
count,0.074861,1.000000


In [99]:
# 실거래가와 아파트가 위치하고 있는 동에서 공원당 평균 면적
pd.concat([y,train_df['park_area_mean']], axis= 1).corr()

,transaction_real_price,park_area_mean
transaction_real_price,1.000000,-0.065479
park_area_mean,-0.065479,1.000000


In [100]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이 집 총 정원 수
pd.concat([y,train_df['day_care_baby_num']], axis= 1).corr()

,transaction_real_price,day_care_baby_num
transaction_real_price,1.000000,0.102403
day_care_baby_num,0.102403,1.000000


In [101]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 보육교직원 수
pd.concat([y,train_df['teacher_num']], axis= 1).corr()

,transaction_real_price,teacher_num
transaction_real_price,1.000,0.018
teacher_num,0.018,1.000


In [102]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 보육실 수
pd.concat([y,train_df['nursing_room_num']], axis= 1).corr()

,transaction_real_price,nursing_room_num
transaction_real_price,1.000000,-0.234696
nursing_room_num,-0.234696,1.000000


In [103]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 놀이터 수 
pd.concat([y,train_df['playground_num']], axis= 1).corr()

,transaction_real_price,playground_num
transaction_real_price,1.000000,-0.228897
playground_num,-0.228897,1.000000


In [104]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 CCTV 설치 수 
pd.concat([y,train_df['CCTV_num']], axis= 1).corr()

,transaction_real_price,CCTV_num
transaction_real_price,1.000000,-0.042957
CCTV_num,-0.042957,1.000000


In [105]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 통학차량 수 
pd.concat([y,train_df['is_commuting_vehicle']], axis= 1).corr()

,transaction_real_price,is_commuting_vehicle
transaction_real_price,1.000000,-0.293391
is_commuting_vehicle,-0.293391,1.000000


In [106]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 수
pd.concat([y,train_df['cnt']], axis= 1).corr()

,transaction_real_price,cnt
transaction_real_price,1.000000,0.070308
cnt,0.070308,1.000000


In [107]:
# 실거래가와 아파트가 위치하고 있는 해당 구의 어린이집 평균 정원 수 
pd.concat([y,train_df['day_care_mean']], axis= 1).corr()

,transaction_real_price,day_care_mean
transaction_real_price,1.000000,-0.001029
day_care_mean,-0.001029,1.000000


In [108]:
# 모델링 하기에 앞서 불필요한 컬럼 제거
train_df = train_df.drop(columns=['transaction_id', 'apartment_id','dong', 'jibun', 'apt',
       'addr_kr', 'transaction_year_month', 'train_YN', 'park_area_mean',
       'gu', 'day_care_mean', 'transaction_real_price', 'apartment_count',
       'apt_id_real_price', 'real_price_mean'])

In [109]:
# 서브미션 파일과 merge를 위해 테스트 셋에서  transaction_id 추출
transaction_id = test_df['transaction_id']

In [110]:
test_df = test_df.drop(columns=['transaction_id', 'apartment_id', 'dong', 'jibun', 'apt',
       'addr_kr', 'transaction_year_month', 'train_YN', 'park_area_mean',
       'gu', 'day_care_mean'])

In [111]:
# 결측값 확인
train_df.isnull().sum()

city                    0
exclusive_use_area      0
year_of_completion      0
transaction_date        0
floor                   0
year                    0
month                   0
quarter                 0
park_area               0
count                   0
day_care_baby_num       0
teacher_num             0
nursing_room_num        0
playground_num          0
CCTV_num                0
is_commuting_vehicle    0
cnt                     0
apartment_price_mean    0
apt_age                 0
dtype: int64

In [112]:
test_df.isnull().sum()

city                      0
exclusive_use_area        0
year_of_completion        0
transaction_date          0
floor                     0
year                      0
month                     0
quarter                   0
park_area                 0
count                     0
day_care_baby_num         0
teacher_num               0
nursing_room_num          0
playground_num            0
CCTV_num                  0
is_commuting_vehicle      0
cnt                       0
apartment_price_mean    136
apt_age                   0
dtype: int64

In [113]:
# 트레인셋에는 있고 테스트셋에는 없는 apartment_id는 0으로 대체
test_df.fillna(0,inplace=True)

# Modeling

### Random Forest

In [114]:
X_forest = train_df

In [115]:
CrossV_forest = pd.concat([X_forest,y],axis=1)

In [116]:
forest_CV = pd.DataFrame(np.zeros(len(test_df)*5).reshape(len(test_df),-1), columns = ['first','second','third','fourth','fifth'])

In [117]:
cv = KFold(5, shuffle=True, random_state=2019) # K-fold = 5 로 진행
for i, (idx_train, idx_test) in enumerate(cv.split(CrossV_forest)):
    forest_train = CrossV_forest.iloc[idx_train]
    forest_test = CrossV_forest.iloc[idx_test]
        
    regressor = RandomForestRegressor(random_state=2019,n_estimators=20) # n_estimator = 20으로 설정 
    result = regressor.fit(forest_train[forest_train.columns[[range(19)]]],forest_train['transaction_real_price']) # 모델 학습
    y_pred_forest = regressor.predict(test_df) # 테스트 파일 넣어서 예측값 산출
    
    forest_CV[forest_CV.columns[i]] = y_pred_forest # 5번의 예측값 평균내기 위해 데이터 프레임 각각 컬럼에 할당

/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py:2095: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py:2095: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)
/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py:2095: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index,

In [118]:
forest_CV.head()

,first,second,third,fourth,fifth
0,37080.0,36892.5,32430.0,35365.0,35065.00
1,29252.5,30167.5,32450.0,34115.0,30285.00
2,36805.0,36387.5,38310.0,33825.0,35282.50
3,37320.0,36360.0,38530.0,34835.0,35462.50
4,25900.0,26957.5,27505.0,26197.5,26203.75


In [119]:
# 5개의 예측값 평균 산출
forest_CV['mean'] = forest_CV.sum(axis=1)/5

In [120]:
# submission 파일과합치기위해 transaction_id 를 concat
id_join_forest = pd.concat([forest_CV,transaction_id],axis=1)

In [121]:
id_join_forest.head()

,first,second,third,fourth,fifth,mean,transaction_id
0,37080.0,36892.5,32430.0,35365.0,35065.00,35366.50,1092791.0
1,29252.5,30167.5,32450.0,34115.0,30285.00,31254.00,1092885.0
2,36805.0,36387.5,38310.0,33825.0,35282.50,36122.00,1092916.0
3,37320.0,36360.0,38530.0,34835.0,35462.50,36501.50,1092919.0
4,25900.0,26957.5,27505.0,26197.5,26203.75,26552.75,1092937.0


In [122]:
# submission dataset 로딩
submission = pd.read_csv("../input/dacon-apartment-price-prediction/submission.csv")

In [123]:
# submission set과 RF(random forest) 결과물과 transaction_id로 merge
final_forest = pd.merge(submission,id_join_forest,on='transaction_id',how='left')

In [124]:
final_forest.head()

,transaction_id,transaction_real_price,first,second,third,fourth,fifth,mean
0,1145756,0,25265.0,31465.0,31685.0,31402.50,31305.0,30224.50
1,1198704,0,13065.0,12635.0,14067.5,13582.75,13040.0,13278.05
2,1222384,0,11787.5,12395.0,11692.5,12190.00,11747.5,11962.50
3,1179897,0,100575.0,106225.0,106860.0,115555.00,115445.0,108932.00
4,1223091,0,46378.7,46875.0,46867.5,46605.00,45922.5,46529.74


In [125]:
# 예측값 5개의 mean 값을 transaction_real_price로 할당
final_forest['transaction_real_price'] = final_forest['mean']

In [126]:
# RF모델로 transaction_id별 예측값 산출
final_real_forest = final_forest[['transaction_id','transaction_real_price']]

In [127]:
final_real_forest.head()

,transaction_id,transaction_real_price
0,1145756,30224.50
1,1198704,13278.05
2,1222384,11962.50
3,1179897,108932.00
4,1223091,46529.74


### Light GBM 

In [128]:
X = train_df
X.head()

,city,exclusive_use_area,year_of_completion,transaction_date,floor,year,month,quarter,park_area,count,day_care_baby_num,teacher_num,nursing_room_num,playground_num,CCTV_num,is_commuting_vehicle,cnt,apartment_price_mean,apt_age
0,1,84.82,2002.0,2.0,2.0,2008.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,53685.538620,6.0
1,1,99.17,1973.0,0.0,6.0,2008.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,58227.693100,35.0
2,1,84.74,2007.0,0.0,6.0,2008.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,56225.207162,1.0
3,1,146.39,2003.0,1.0,15.0,2008.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,107829.594420,5.0
4,1,194.43,2004.0,2.0,3.0,2008.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117395.179540,4.0


In [129]:
CrossV_lgbm = pd.concat([X,y],axis=1)

In [130]:
# 파라미터 값 지정
parameters = {
    'objective': 'regression_l2', 
    'metric': 'rmse', #loss 측정기준
    'is_unbalance': 'false',
    'boosting': 'gbdt', #gbdt(gradient boosting decision tree) 사용
    'num_leaves': 50,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.8, 
    'bagging_freq': 40,
    'learning_rate': 0.05,
    'min_data' : 500,
    'verbose': 1
}

In [131]:
# Light GBM도 마찬가지로 Cross validation 진행

lgbm_CV = pd.DataFrame(np.zeros(len(test_df)*5).reshape(len(test_df),-1), columns = ['first','second','third','fourth','fifth'])
cv = KFold(5, shuffle=True, random_state=2019) # K-fold =5 진행
for i, (idx_train, idx_test) in enumerate(cv.split(CrossV_lgbm)):
    df_train = CrossV_lgbm.iloc[idx_train]
    df_test = CrossV_lgbm.iloc[idx_test]
    
    train_data = lgb.Dataset(df_train[df_train.columns[[range(19)]]],label=df_train[df_train.columns[19]])
    test_data = lgb.Dataset(df_test[df_test.columns[[range(19)]]],label=df_test[df_test.columns[19]])
    
    model = lgb.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=7000, # 부스팅횟수 : 7000
                       early_stopping_rounds=100, verbose_eval=200)
    pred_y = model.predict(test_df)
    
    lgbm_CV[lgbm_CV.columns[i]] = pred_y

/opt/conda/lib/python3.6/site-packages/pandas/core/indexes/base.py:2095: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  result = getitem(key)


Training until validation scores don't improve for 100 rounds.
[200]	valid_0's rmse: 6133.18
[400]	valid_0's rmse: 5523.37
[600]	valid_0's rmse: 5227.26
[800]	valid_0's rmse: 5054.79
[1000]	valid_0's rmse: 4926.61
[1200]	valid_0's rmse: 4832
[1400]	valid_0's rmse: 4748.58
[1600]	valid_0's rmse: 4677.36
[1800]	valid_0's rmse: 4627.03
[2000]	valid_0's rmse: 4571.35
[2200]	valid_0's rmse: 4525.52
[2400]	valid_0's rmse: 4486.73
[2600]	valid_0's rmse: 4450.45
[2800]	valid_0's rmse: 4420.22
[3000]	valid_0's rmse: 4389.57
[3200]	valid_0's rmse: 4360.97
[3400]	valid_0's rmse: 4340.56
[3600]	valid_0's rmse: 4317.06
[3800]	valid_0's rmse: 4291.57
[4000]	valid_0's rmse: 4272.94
[4200]	valid_0's rmse: 4253.65
[4400]	valid_0's rmse: 4232.67
[4600]	valid_0's rmse: 4217.36
[4800]	valid_0's rmse: 4198.85
[5000]	valid_0's rmse: 4184.31
[5200]	valid_0's rmse: 4170.29
[5400]	valid_0's rmse: 4161.94
[5600]	valid_0's rmse: 4152.01
[5800]	valid_0's rmse: 4141.2
[6000]	valid_0's rmse: 4132.53
[6200]	valid_0'

In [132]:
lgbm_CV.head()

,first,second,third,fourth,fifth
0,31464.499399,31708.188330,32240.789008,31739.200228,31825.683286
1,32149.193427,31970.106421,31914.680556,32044.426059,32490.420087
2,35118.641949,34749.026294,34883.809445,35541.813303,34820.668828
3,33826.087487,33592.123207,33754.458445,34611.239456,33932.150965
4,26283.408768,26310.487920,26102.261664,26775.669029,26461.807259


In [133]:
# 5개의 예측값 평균 산출
lgbm_CV['mean'] = lgbm_CV.sum(axis=1)/5 

In [134]:
# submission 파일과합치기위해 transaction_id 를 concat
id_join_lgbm = pd.concat([lgbm_CV,transaction_id],axis=1)

In [135]:
id_join_lgbm.head()

,first,second,third,fourth,fifth,mean,transaction_id
0,31464.499399,31708.188330,32240.789008,31739.200228,31825.683286,31795.672050,1092791.0
1,32149.193427,31970.106421,31914.680556,32044.426059,32490.420087,32113.765310,1092885.0
2,35118.641949,34749.026294,34883.809445,35541.813303,34820.668828,35022.791964,1092916.0
3,33826.087487,33592.123207,33754.458445,34611.239456,33932.150965,33943.211912,1092919.0
4,26283.408768,26310.487920,26102.261664,26775.669029,26461.807259,26386.726928,1092937.0


In [136]:
submission = pd.read_csv('../input/dacon-apartment-price-prediction/submission.csv')

In [137]:
final_lgbm = pd.merge(submission,id_join_lgbm,on='transaction_id',how='outer')

In [138]:
# 예측값 5개의 mean(평균) 값을 transaction_real_price로 할당
final_lgbm['transaction_real_price'] = final_lgbm['mean']

In [139]:
final_real_lgbm = final_lgbm[final_lgbm.columns[[0,1]]]

In [140]:
final_real_lgbm.head()

,transaction_id,transaction_real_price
0,1145756,26046.619294
1,1198704,16734.892447
2,1222384,11467.711975
3,1179897,110797.169418
4,1223091,48357.971616


In [141]:
# RF 모델과 Lgbm 모델의 예측값들을 합침
real_final = pd.merge(final_real_forest,final_real_lgbm,on='transaction_id',how='outer')

In [142]:
# 두 모델의 예측값의 평균값을 최종 예측값으로 사용
real_final['transaction_real_price'] = (real_final['transaction_real_price_x']+real_final['transaction_real_price_y'])/2

In [143]:
real_finall = real_final[['transaction_id','transaction_real_price']]

In [144]:
real_finall.to_csv('submission_ensemble.csv',index=False)